In [1]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
from matplotlib import cm

# ============================
# LOAD MODEL (FORCE BUILD)
# ============================
def load_model_with_init(model_path):
    model = tf.keras.models.load_model(
        model_path,
        custom_objects={'Adam': tf.keras.optimizers.Adam, 'AUC': tf.keras.metrics.AUC}
    )
    # Force model to build (important for Grad-CAM)
    dummy_input = tf.ones((1, 224, 224, 3))
    _ = model.predict(dummy_input, verbose=0)
    return model

try:
    model = load_model_with_init('medical_image_classifier_3class.h5')
    print("Model successfully loaded and initialized!")
except Exception as e:
    print(f"Failed to load model: {e}")
    raise

class_names = ['Normal', 'Pneumonia Bacterial', 'Pneumonia Viral']

# ============================
# PREPROCESS IMAGE
# ============================
def preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# ============================
# GRAD-CAM HEATMAP
# ============================
def make_gradcam_heatmap(img_array, model, last_conv_layer_name='block3_conv2'):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy(), pred_index.numpy()

# ============================
# CREATE HEATMAP OVERLAY
# ============================
def create_heatmap_overlay(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    heatmap = np.uint8(255 * heatmap)
    jet = cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img * (1 - alpha)
    return tf.keras.preprocessing.image.array_to_img(superimposed_img)

# ============================
# MAIN PREDICTION FUNCTION
# ============================
def predict_image(img):
    temp_path = "temp_img.png"
    img.save(temp_path)

    try:
        processed = preprocess_image(temp_path)
        preds = model.predict(processed, verbose=0)[0]

        heatmap, pred_index = make_gradcam_heatmap(processed, model)
        heatmap_img = create_heatmap_overlay(temp_path, heatmap)

        heatmap_path = "heatmap.png"
        heatmap_img.save(heatmap_path)

        plt.figure(figsize=(10, 4))
        bars = plt.barh(class_names, preds)
        plt.title('Probabilidades de Clasificacion', fontsize=12)
        plt.xlabel('Probabilidad')
        plt.xlim(0, 1)
        for bar in bars:
            width = bar.get_width()
            plt.text(width + 0.02, bar.get_y() + bar.get_height()/2,
                     f'{width:.1%}', ha='left', va='center', fontsize=9)
        plt.tight_layout()
        prob_plot_path = 'probabilities.png'
        plt.savefig(prob_plot_path, bbox_inches='tight', dpi=100)
        plt.close()

        return (
            class_names[pred_index], 
            f"{np.max(preds):.1%}",
            prob_plot_path,
            pd.DataFrame({'Class': class_names, 'Probabilidad': preds}),
            heatmap_path
        )
    except Exception as e:
        print(f"Error de prediccion: {e}")
        return (
            "Error", "0%", None,
            pd.DataFrame({'Class': class_names, 'Probability': [0.33]*3}),
            None
        )
    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)

# ============================
# GRADIO INTERFACE
# ============================
with gr.Blocks(title="Clasificador de Neumonias", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""# Clasificador de Neumonias a partir de Radiografias""")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Radiografia", height=300)
            submit_btn = gr.Button("Clasificar", variant="primary")
        
        with gr.Column():
            diagnosis = gr.Textbox(label="Diagnostico")
            confidence = gr.Textbox(label="Confianza")
            prob_plot = gr.Image(label="Probabilidades", height=200)

    with gr.Row():
        with gr.Column():
            bar_plot = gr.BarPlot(
                pd.DataFrame({'Class': class_names, 'Probabilidad': [0.33]*3}),
                x="Class", y="Probabilidad", title="Probabilidades de Clases",
                height=300, width=400
            )
        with gr.Column():
            gradcam_output = gr.Image(label="Grad-CAM Heatmap", height=300)

    submit_btn.click(
        predict_image,
        inputs=image_input,
        outputs=[diagnosis, confidence, prob_plot, bar_plot, gradcam_output]
    )

    gr.Markdown("""
    <div style='background-color: #f8f9fa; padding: 15px; border-radius: 5px;'>
    <small><b>Disclaimer:</b> For research/educational use only. Not for clinical diagnosis.</small>
    </div>
    """)

if __name__ == "__main__":
    demo.launch(share=True)

2025-07-27 16:21:01.732850: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-27 16:21:01.741001: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-27 16:21:01.823475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-27 16:21:01.895939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753654861.967437    5119 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753654861.98

Model successfully loaded and initialized!
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4542b8c341bb9a094c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/lugo/.local/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)
libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
/tmp/ipykernel_5119/1532691814.py:75: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  jet = cm.get_cmap("jet")
/home/lugo/.local/lib/python3.10/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)
/tmp/ipykernel_5119/1532691814.py:75: MatplotlibDeprecationWarning: The get_cmap function was deprecated in 